In [100]:
from sklearn import linear_model,cross_validation, preprocessing
import numpy as np

# Read the two data file

trainDataMatrix = []
trainDataLabel = []
trainfp = open('train.csv')
with trainfp as fr:
    next(fr)
    for line in fr:
        temp = line.strip().split(',')
        trainDataLabel.append(temp[1])
        trainDataMatrix.append(temp[2:])

testDataMatrix = []
testID =[]
testfp = open('test.csv')
with testfp as fr:
    next(fr)
    for line in fr:
        temp = line.strip().split(',')
        testDataMatrix.append(temp[1:])
        testID.append(temp[0])
# return trainDataMatrix,trainDataLabel
# print (testID)

# Eliminate irrelevant data：
# Ticket class, Name, ticket number

for i in range(0,len(trainDataMatrix)):
    # Delete all the names
    # first & last
    del trainDataMatrix[i][2]
    del trainDataMatrix[i][1]
    # change male/female to numbers
    if trainDataMatrix[i][1]=="male":
        trainDataMatrix[i][1] = 1
    elif trainDataMatrix[i][1]=="female":
        trainDataMatrix[i][1] = 0
    # Delete ticket number
    del trainDataMatrix[i][5]
    del trainDataMatrix[i][6]
    # change male/female to numbers
    if trainDataMatrix[i][6]=="C":
        trainDataMatrix[i][6] = 1
    elif trainDataMatrix[i][6]=="S":
        trainDataMatrix[i][6] = 2
    elif trainDataMatrix[i][6]=="Q":
        trainDataMatrix[i][6] = 3
    elif trainDataMatrix[i][6]=="":
        trainDataMatrix[i][6] = -1
    # if age is missing change it with -1
    if trainDataMatrix[i][2]=="":
        trainDataMatrix[i][2] = -1

for i in range(0,len(testDataMatrix)):
    # Delete all the names
    del testDataMatrix[i][2]
    del testDataMatrix[i][1]
    # change male/female to numbers
    if testDataMatrix[i][1]=="male":
        testDataMatrix[i][1] = 1
    elif testDataMatrix[i][1]=="female":
        testDataMatrix[i][1] = 0
    # Delete ticket number
    del testDataMatrix[i][5]
    del testDataMatrix[i][6]
    # change male/female to numbers
    if testDataMatrix[i][6]=="C":
        testDataMatrix[i][6] = 1
    elif testDataMatrix[i][6]=="S":
        testDataMatrix[i][6] = 2
    elif testDataMatrix[i][6]=="Q":
        testDataMatrix[i][6] = 3
    elif testDataMatrix[i][6]=="":
        testDataMatrix[i][6] = -1
    # if age is missing change it with -1
    if testDataMatrix[i][2]=="":
        testDataMatrix[i][2] = -1
    if testDataMatrix[i][5]=="":
        testDataMatrix[i][5] = -1
    testDataMatrix[i] = np.array(testDataMatrix[i]).astype(float)        

# # return trainDataMatrix,trainDataLabel
trainDataLabel = np.array(trainDataLabel).astype(float)
trainDataMatrix = np.array(trainDataMatrix).astype(float)
# 0: pclass
# 1: sex
# 2: age
# 3: sib/spouse
# 4: parents/children
# 5: Fare
# 6: Embarked

# print (trainDataMatrix)
# print (testDataMatrix)

[[  3.       1.      22.     ...,   0.       7.25     2.    ]
 [  1.       0.      38.     ...,   0.      71.2833   1.    ]
 [  3.       0.      26.     ...,   0.       7.925    2.    ]
 ..., 
 [  3.       0.      -1.     ...,   2.      23.45     2.    ]
 [  1.       1.      26.     ...,   0.      30.       1.    ]
 [  3.       1.      32.     ...,   0.       7.75     3.    ]]


The following used 10-fold to predict the score of the classifier.

b) The following features are taken into consideration:
0: pclass : the class of the cabin may determine if the lifeboats are closer or not
1: sex : males are more likely to give the chance to woman as shown in the movie
2: age : the old will be more likely to give the chance to the young people as they are going to die soon anyway
3: sib/spouse: families are running as a group, they don't abandom the others
4: parents/children: Similarly as sib/spouse case
5: Fare: people who has priviledge or intimate relationship with the system could have lower fare to get on ship as they are more likely to get the liveboat 
6: Embarked: different on board location may determine their attitude toward whether to live or give the chance to the others

In [116]:
import csv
trainedModel = linear_model.LogisticRegression()
def evaluate(*fold):
    trainedModel = linear_model.LogisticRegression()
    train_indices, test_indices= fold
    X_train = trainDataMatrix[train_indices]
    Y_train = trainDataLabel[train_indices]
    X_test = trainDataMatrix[test_indices]
    Y_test = trainDataLabel[test_indices]
    
    scaler = preprocessing.StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    x = trainedModel.fit(X_train,Y_train).score(X_test,Y_test)
    print (x)
    return trainedModel.fit(X_train,Y_train)

for train, test in (cross_validation.KFold(len(trainDataLabel), n_folds=10)):
    trainedModel = evaluate(train, test)


    

scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(trainDataMatrix)
X_test = scaler.fit_transform(testDataMatrix)
# 0: pclass
# 1: sex
# 2: age
# 3: sib/spouse
# 4: parents/children
# 5: Fare
# 6: Embarked

trainedModel =  trainedModel.fit(X_train,trainDataLabel)
answer =   trainedModel.predict(X_test).astype(int)        
# print (trainedModel)


with open('answer.csv', 'w') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    csvfile.write("PassengerId,Survived\n")
    for i in range(len(testID)):
        tempStr = ""+str(testID[i]) + str(answer[i])
        spamwriter.writerow([str(testID[i]),str(answer[i])])


0.8
0.820224719101
0.741573033708
0.797752808989
0.752808988764
0.76404494382
0.76404494382
0.719101123596
0.865168539326
0.797752808989
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


c) Kaggle result:
Name        Submitted       Wait time     Execution time    Score
answer.csv  0 minutes ago   0 seconds     0 seconds         0.76076
